## KOCW 데이터베이스 시연(`R`)

In [13]:
library(DBI)
library(RSQLite)
library(readxl)

In [14]:
conn = dbConnect(SQLite(), "kocw.db")

In [15]:
dbListTables(conn)

[1] "charge"         "course_classed" "norm_course"    "professor"     
[5] "university"

In [16]:
dbExecute(conn, "PRAGMA foreign_keys = ON")

[1] 0

### 1. 기본키 설정이 맞는지 확인 + 튜플의 수정 및 삭제

`-` 중복되지 않는 기본키를 가지는 튜플 삽입 시(정상조작)

In [33]:
## professor 중복되지 않는 기본키를 가지는 튜플 삽입 시 정상
dbExecute(conn, "INSERT INTO professor VALUES(-1, '강신성', '자연계열', '전북대학교')")
dbGetQuery(conn, "SELECT * FROM professor LIMIT 5")

## 수정
dbExecute(conn, "UPDATE professor SET 제공대학명 = '1조대학교' WHERE 교수코드 = -1")
dbGetQuery(conn, "SELECT * FROM professor LIMIT 5")

## 삭제
dbExecute(conn, "DELETE FROM professor WHERE 교수코드 = -1")
dbGetQuery(conn, "SELECT * FROM professor LIMIT 5")

[1] 1

교수코드,담당교수명,담당분야,제공대학명
<int>,<chr>,<chr>,<chr>
-1,강신성,자연계열,전북대학교
0,안상욱,자연계열,한경대학교
1,이중호,자연계열,가천대학교
2,최수한,자연계열,단국대학교
3,김문현,자연계열,성균관대학교


[1] 1

교수코드,담당교수명,담당분야,제공대학명
<int>,<chr>,<chr>,<chr>
-1,강신성,자연계열,1조대학교
0,안상욱,자연계열,한경대학교
1,이중호,자연계열,가천대학교
2,최수한,자연계열,단국대학교
3,김문현,자연계열,성균관대학교


[1] 1

교수코드,담당교수명,담당분야,제공대학명
<int>,<chr>,<chr>,<chr>
0,안상욱,자연계열,한경대학교
1,이중호,자연계열,가천대학교
2,최수한,자연계열,단국대학교
3,김문현,자연계열,성균관대학교
4,이상준,자연계열,덕성여자대학교


In [50]:
## university 중복되지 않은 기본키를 가지는 튜플 삽입 시 정상
dbExecute(conn, "INSERT INTO university VALUES('1조대학교', '뉴욕', '사립')")
dbGetQuery(conn, "SELECT * FROM university LIMIT 10 OFFSET 182")
dbExecute(conn, "DELETE FROM university WHERE 제공대학명 = '1조대학교'")

[1] 1

제공대학명,대학소재지,구분
<chr>,<chr>,<chr>
사이버한국외국어대학교,서울특별시,사립
수원대학교,경기도,사립
구미대학교,경상북도,사립
가톨릭관동대학교,강원특별자치도,사립
1조대학교,뉴욕,사립


[1] 1

`-` 중복되는 기본키를 가지는 튜플 삽입 시(비정상 조작)

In [36]:
## 중복되는 기본키를 가지는 튜플 삽입 시 오류 발생 -> 정상적으로 기본키 설정됨
dbExecute(conn, "INSERT INTO professor VALUES(1, '강신성2', '자연계열', '전북대학교')")

ERROR: Error: UNIQUE constraint failed: professor.교수코드


In [46]:
## 중복되는 기본키를 가지는 튜플 삽입 시 오류 발생 -> 정상적으로 기본키 설정됨
dbExecute(conn, "INSERT INTO university VALUES('전북대학교', '뉴욕', '국립')")

ERROR: Error: UNIQUE constraint failed: university.제공대학명


### 2. 기본키 삭제에 따른 외래키의 변화 확인

`-` 외래키가 있는 릴레이션의 CASCADE / SET NULL 기준 서술

1. 담당교수 릴레이션

   * 소속대학명 `ON UPDATE CASCADE, ON DELETE SET NULL`

   > UPDATE : 소속된 대학의 이름이 바뀌어도 교수가 그 대학에 소속되었다는 사실은 변하지 않음.
   >
   > DELETE : 소속된 대학이 사라져도 교수는 강의를 제공했기 때문에 교수에 대한 정보를 유지하고자 함.

3. 강의 릴레이션

    * 제공대학명 `ON UPDATE CASCADE, ON DELETE SET NULL`
  
   > UPDATE : 제공한 대학의 이름이 바뀌어도 그 대학이 강의를 제공했었다는 사실은 변하지 않음
   >
   > DELETE : 제공한 대학이 사라져도 제공되었던 강의 정보를 유지하고자 함.

    * 강의명 `ON UPDATE CASCADE, ON DELETE CASCADE`
  
    > UPDATE : 기존에 있었던 강의명이 바뀌어도 다른 속성값들은 그대로임.
    >
    > DELETE : 강의명을 없앤다는 것 자체의 의도가 해당 강의를 없애고 싶다는 의미이므로 같이 삭제하는 게 합리적.

3. 담당 릴레이션

    * 교수코드 및 강의코드 `ON UPDATE CASCADE, ON DELETE CASCADE`
  
   > UPDATE : 교수코드 및 강의코드가 수정되어도 매핑 정보를 유지해줘야 함.
   >
   > DELETE : 삭제했을 시 남겨둬야 할 정보가 없음 & 기본키에 NULL이 존재하면 개체 무결성 제약 깨짐

In [5]:
dbGetQuery(conn, "SELECT * FROM professor WHERE 제공대학명 = '전북대학교' LIMIT 5")

교수코드,담당교수명,담당분야,제공대학명
<int>,<chr>,<chr>,<chr>
9,마인숙,자연계열,전북대학교
103,곽지이,자연계열,전북대학교
104,이선영,자연계열,전북대학교
105,정수미,자연계열,전북대학교
106,최일선,자연계열,전북대학교


In [6]:
dbGetQuery(conn, "SELECT * FROM norm_course WHERE 제공대학명 = '전북대학교' LIMIT 5")

강의코드,강의명,강의년도,강의학기,제공일자,제공대학명
<int>,<chr>,<int>,<int>,<chr>,<chr>
133761,자바 프로그래밍,2009,1,2008-12-11,전북대학교
145685,조경수 관리,2009,1,2009-11-05,전북대학교
153670,전력전자 및 실험,2009,2,2010-04-28,전북대학교
156266,기능생리학 1,2009,1,2010-06-10,전북대학교
156446,BIN융합개론및세미나 1,2009,1,2010-06-11,전북대학교


In [7]:
dbGetQuery(conn, "SELECT * FROM university WHERE 제공대학명 = '전북대학교'")

제공대학명,대학소재지,구분
<chr>,<chr>,<chr>
전북대학교,전라북도,국립


In [8]:
dbGetQuery(conn, "SELECT * FROM norm_course WHERE 제공대학명 = '전북대학교' LIMIT 5")

강의코드,강의명,강의년도,강의학기,제공일자,제공대학명
<int>,<chr>,<int>,<int>,<chr>,<chr>
133761,자바 프로그래밍,2009,1,2008-12-11,전북대학교
145685,조경수 관리,2009,1,2009-11-05,전북대학교
153670,전력전자 및 실험,2009,2,2010-04-28,전북대학교
156266,기능생리학 1,2009,1,2010-06-10,전북대학교
156446,BIN융합개론및세미나 1,2009,1,2010-06-11,전북대학교


In [9]:
dbExecute(conn, "UPDATE university SET 제공대학명 = 'ㅁㄴㅇㄹ' WHERE 제공대학명 = '전북대학교'")
dbGetQuery(conn, "SELECT * FROM norm_course WHERE 제공대학명 = '전북대학교' LIMIT 5")
dbExecute(conn, "UPDATE university SET 제공대학명 = '전북대학교' WHERE 제공대학명 = '전북특별자치대학교'")

ERROR: Error: FOREIGN KEY constraint failed


In [31]:
dbGetQuery(conn, "SELECT * FROM university WHERE 제공대학명 = '전북대학교'")

제공대학명,대학소재지,구분
<chr>,<chr>,<chr>
전북대학교,전라북도,국립


In [15]:
dbGetQuery(conn, "SELECT * FROM sqlite_schema WHERE name = 'professor'")
dbGetQuery(conn, "SELECT * FROM sqlite_schema WHERE name = 'norm_course'")
dbGetQuery(conn, "SELECT * FROM sqlite_schema WHERE name = 'charge'")

type,name,tbl_name,rootpage,sql
<chr>,<chr>,<chr>,<int>,<chr>
table,professor,professor,5,"CREATE TABLE professor ( 교수코드 INTEGER PRIMARY KEY, 담당교수명 TEXT, 담당분야 TEXT, 제공대학명 TEXT, FOREIGN KEY (제공대학명) REFERENCES university(제공대학명) ON UPDATE CASCADE ON DELETE SET NUll )"


type,name,tbl_name,rootpage,sql
<chr>,<chr>,<chr>,<int>,<chr>
table,norm_course,norm_course,4,"CREATE TABLE norm_course ( 강의코드 INTEGER PRIMARY KEY, 강의명 TEXT, 강의년도 INTEGER, 강의학기 INTEGER, 제공일자 TEXT, 제공대학명 TEXT, FOREIGN KEY (제공대학명) REFERENCES university(제공대학명) )"


type,name,tbl_name,rootpage,sql
<chr>,<chr>,<chr>,<int>,<chr>
table,charge,charge,6,"CREATE TABLE charge ( 교수코드 INTEGER, 강의코드 INTEGER, FOREIGN KEY (교수코드) REFERENCES professor(교수코드) ON UPDATE CASCADE ON DELETE CASCADE FOREIGN KEY (강의코드) REFERENCES norm_course(강의코드) ON UPDATE CASCADE ON DELETE CASCADE PRIMARY KEY (교수코드, 강의코드) )"


In [11]:
dbGetQuery(conn, "SELECT * FROM sqlite_schema WHERE name = 'university'")

type,name,tbl_name,rootpage,sql
<chr>,<chr>,<chr>,<int>,<chr>
table,university,university,2,"CREATE TABLE university ( 제공대학명 TEXT PRIMARY KEY, 대학소재지 TEXT, 구분 TEXT )"


In [ ]:
dbExecute(conn, "INSERT")

In [10]:
dbExecute(conn, "DELETE FROM professor WHERE 교수코드 = 0")

[1] 10

In [17]:
dbGetQuery(conn, "SELECT * FROM charge LIMIT 20")

교수코드,강의코드
<int>,<int>
0,1095582
1,1104074
2,1110603
3,1112400
4,1118072
4,1118158
5,1118268
6,1123826
7,1125165


In [13]:
dbGetQuery(conn, "SELECT * FROM sqlite_schema WHERE name = 'charge'")

type,name,tbl_name,rootpage,sql
<chr>,<chr>,<chr>,<int>,<chr>
table,charge,charge,2,"CREATE TABLE charge ( 교수코드 INTEGER, 강의코드 INTEGER, PRIMARY KEY (교수코드, 강의코드), FOREIGN KEY (교수코드) REFERENCES professor(교수코드), FOREIGN KEY (강의코드) REFERENCES norm_course(강의코드) )"


In [1]:
professor_data <- read_excel("C:/Users/leesj/Downloads/professor (2).xlsx")
course_classed_data <- read_excel("C:/Users/leesj/Downloads/course_classed (2).xlsx")
norm_course_data <- read_excel("C:/Users/leesj/Downloads/norm_course (3).xlsx")
charge_data <- read_excel("C:/Users/leesj/Downloads/charge (3).xlsx")
university_data <- read_excel("C:/Users/leesj/Downloads/university (3).xlsx")

professor_data <- professor_data[,-1]
course_classed_data <- course_classed_data[,-1]
norm_course_data <- norm_course_data[,-1]
charge_data <- charge_data[,-1]
university_data <- university_data[,-1]

names(university_data)[names(university_data) == "대학소재지(시/도)"] <- "대학소재지"
names(university_data)[names(university_data) == "사립/국립 구분"] <- "구분"

conn <- dbConnect(RSQLite::SQLite(), "kocw.db")

dbExecute(conn, "PRAGMA foreign_keys = ON")

dbExecute(conn, "
CREATE TABLE charge (
  FOREIGN KEY (교수코드) 
  REFERENCES professor(교수코드)
  ON UPDATE SET CASCADE
  ON DELECT SET NULL,
  FOREIGN KEY (강의코드) 
  REFERENCES norm_course(강의코드)
  ON UPDATE CASCADE
  ON DELECT SET NULL,
  PRIMARY KEY (교수코드, 강의코드)
)")

dbExecute(conn, "
CREATE TABLE norm_course (
  강의코드 INTEGER PRIMARY KEY,
  강의명 TEXT,
  강의년도 INTEGER,
  강의학기 INTEGER,
  제공대학명 TEXT,
  제공일자 TEXT,
  FOREIGN KEY (제공대학명) 
  REFERENCES university(제공대학명)
)")

dbExecute(conn, "
CREATE TABLE course_classed (
  강의명 TEXT PRIMARY KEY,
  교육분류 TEXT
)")

dbExecute(conn, "
CREATE TABLE professor (
  교수코드 INTEGER PRIMARY KEY,
  담당교수명 TEXT,
  담당분야 TEXT,
  제공대학명 TEXT,
  FOREIGN KEY (제공대학명)
  REFERENCES university(제공대학명)
)")

dbExecute(conn, "
CREATE TABLE university (
  제공대학명 TEXT PRIMARY KEY,
  대학소재지 TEXT,
  구분 TEXT
)")

dbWriteTable(conn, "university", university_data, append = TRUE, row.names = FALSE)
dbWriteTablea(conn, "professor", professor_data, append = TRUE, row.names = FALSE)
dbWriteTable(conn, "norm_course", norm_course_data, append = TRUE, row.names = FALSE)
dbWriteTable(conn, "course_classed", course_classed_data, append = TRUE, row.names = FALSE)
dbWriteTable(conn, "charge", charge_data, append = TRUE, row.names = FALSE)


dbListTables(conn)



dbDisconnect(conn)

ERROR: Error in read_excel("C:/Users/leesj/Downloads/professor (2).xlsx"): could not find function "read_excel"
